In [10]:
##移除异常点
##划分测试集

import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt

df = pd.read_csv('../dataSets/training/training_20min_avg_travel_time.csv')

## 计算volume

In [9]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

df = pd.read_csv('dataSets/training/training_20min_avg_volume.csv')


volume_in = {}
volume_out = {}
start_time  = datetime.strptime("2016-09-19 00:00:00",'%Y-%m-%d %H:%M:%S')

for idx in range(df.shape[0]):
    line = df.iloc[idx]
    tollgate = str(line['tollgate_id'])
    stime = datetime.strptime(line['time_window'][1:20],'%Y-%m-%d %H:%M:%S')
    day = (stime - start_time).days
    tw_n = (stime.hour * 60 + stime.minute)/20
    
    if line['direction'] == 0: ##判断进/出流量
        if tollgate not in volume_in.keys():
            volume_in[tollgate] = np.zeros(72*29).reshape(29,72)
        volume_in[tollgate][day][tw_n] = line['volume']
    else:
        if tollgate not in volume_out.keys():
            volume_out[tollgate] = np.zeros(72*29).reshape(29,72)
        volume_out[tollgate][day][tw_n] = line['volume']

IOError: File dataSets/training/training_20min_avg_volume.csv does not exist

In [69]:
## 计算均值
mean_volume_in = {}
mean_volume_out = {}
for tollgate_id in volume_in.keys():
    totalvolume = volume_in[tollgate_id]
    mean_volume_in[tollgate_id] = np.mean(totalvolume,axis=0)
    
for tollgate_id in volume_out.keys():
    totalvolume = volume_out[tollgate_id]
    mean_volume_out[tollgate_id] = np.mean(totalvolume,axis=0)

In [78]:
mean_volume_in.keys()

['1', '3', '2']

In [83]:
## 输出均值
## 输出格式：对于每个收费站，输出每一天的同一个时间窗的流量。然后每一天第二个时间窗
import pandas as pd

tollgate = []
time_window=[]
direction = []
volume = []

n_day = 7
for tollgate_id in ['1','2','3']:
    agv_volume_in = mean_volume_in[tollgate_id]
    
    ## 上午进入流量
    i = 0
    for k in [24,25,26,27,28,29]:
        start_time = datetime.strptime('2016-10-18 08:00:00',"%Y-%m-%d %H:%M:%S")
        for day in range(n_day):
            starttime = start_time + timedelta(days=day)
            tollgate.append(tollgate_id)
            time_window.append('\"[' + (starttime + timedelta(seconds=1200*i)).strftime("%Y-%m-%d %H:%M:%S")\
                               + "," + (starttime + timedelta(seconds=1200*i+1200)).strftime("%Y-%m-%d %H:%M:%S") + ')\"')
            direction.append(0)
            volume.append(agv_volume_in[k])
        i+=1
    
    ## 上午流出流量
    if tollgate_id in mean_volume_out.keys():
        agv_volume_out = mean_volume_out[tollgate_id]
        i = 0
        for k in [24,25,26,27,28,29]:
            start_time = datetime.strptime('2016-10-18 08:00:00',"%Y-%m-%d %H:%M:%S")
            for day in range(n_day):
                starttime = start_time + timedelta(days=day)
                tollgate.append(tollgate_id)
                time_window.append('\"[' + (starttime + timedelta(seconds=1200*i)).strftime("%Y-%m-%d %H:%M:%S")\
                                   + "," + (starttime + timedelta(seconds=1200*i+1200)).strftime("%Y-%m-%d %H:%M:%S") + ')\"')
                direction.append(1)
                volume.append(agv_volume_out[k])
            i+=1

 
for tollgate_id in ['1','2','3']:
    agv_volume_in = mean_volume_in[tollgate_id]
    
    ## 下午进入流量
    i = 0
    for k in [51,52,53,54,55,56]:
        start_time = datetime.strptime('2016-10-18 17:00:00',"%Y-%m-%d %H:%M:%S")
        for day in range(n_day):
            starttime = start_time + timedelta(days=day)
            tollgate.append(tollgate_id)
            time_window.append('\"[' + (starttime + timedelta(seconds=1200*i)).strftime("%Y-%m-%d %H:%M:%S")\
                               + "," + (starttime + timedelta(seconds=1200*i+1200)).strftime("%Y-%m-%d %H:%M:%S") + ')\"')
            direction.append(0)
            volume.append(agv_volume_in[k])
        i+=1
    
    ## 下午流出流量
    if tollgate_id in mean_volume_out.keys():
        agv_volume_out = mean_volume_out[tollgate_id]
        i = 0
        for k in [51,52,53,54,55,56]:
            start_time = datetime.strptime('2016-10-18 17:00:00',"%Y-%m-%d %H:%M:%S")
            for day in range(n_day):
                starttime = start_time + timedelta(days=day)
                tollgate.append(tollgate_id)
                time_window.append('\"[' + (starttime + timedelta(seconds=1200*i)).strftime("%Y-%m-%d %H:%M:%S")\
                                   + "," + (starttime + timedelta(seconds=1200*i+1200)).strftime("%Y-%m-%d %H:%M:%S") + ')\"')
                direction.append(1)
                volume.append(agv_volume_out[k])
            i+=1

d = {"tollgate_id":tollgate,"time_window":time_window,"direction":direction,"volume":volume}
pd = DataFrame(data=d)
pd.to_csv('volume.csv',index=False,columns=["tollgate_id","time_window","direction","volume"])